In [ ]:
import pandas as pd 

data_dir = '../data/'

animation_df = pd.read_csv(data_dir + 'kuka_2.csv')
animation_df.head()

In [ ]:
print(f"frame df length: {len(animation_df):}")

In [ ]:
filtered_df = animation_df[~animation_df['Link'].str.contains('Axis')]
filtered_df.head()
print(f"filtered df length: {len(filtered_df):}")


In [ ]:
links = ["LinkN0", "LinkN1", "LinkN2", "LinkN3", "LinkN4", "LinkN5", "LinkN6", "LinkN7"]
weird_data = filtered_df[~filtered_df['Link'].isin(links)]
print(f"link names: {weird_data['Link'].unique()}")

In [18]:
weird_link = "LinkN6.002"
print(f"number of rows containing {weird_link}: {len(filtered_df[filtered_df['Link'] == weird_link])}")

number of rows containing LinkN6.002: 461


In [19]:
cleaned_df = filtered_df[~filtered_df['Link'].str.contains(weird_link)]
print(len(cleaned_df))

3688


In [20]:
print(f"unique link names before cleaning: {animation_df['Link'].unique()}")
print(f"unique link names after cleaning: {cleaned_df['Link'].unique()}")


unique link names before cleaning: ['LinkN0' 'LinkN0-Axis' 'LinkN1' 'LinkN1-Axis' 'LinkN2' 'LinkN2-Axis'
 'LinkN3' 'LinkN3_Axis' 'LinkN4' 'LinkN4_Axis' 'LinkN5' 'LinkN5_Axis'
 'LinkN6' 'LinkN6.002' 'LinkN7']
unique link names after cleaning: ['LinkN0' 'LinkN1' 'LinkN2' 'LinkN3' 'LinkN4' 'LinkN5' 'LinkN6' 'LinkN7']


In [ ]:
link_dfs = {}

for link in links:
    link_dfs[link] = cleaned_df[cleaned_df['Link'] == link].reset_index(drop=True)
    if link != "LinkN7":
        link_dfs[link]['qpos'] = link_dfs[link]['Y_Rotation']
        link_dfs[link]['qvel'] = link_dfs[link]['Y_Velocity']
    else:
        link_dfs[link]['qpos'] = link_dfs[link]['Z_Rotation'] 
        link_dfs[link]['qvel'] = link_dfs[link]['Z_Velocity']
        
    print(f"Created dataframe for {link} with {len(link_dfs[link])} rows")



In [ ]:
link_dfs['LinkN6']

In [23]:
# Start with the first link's frame, qpos and qvel
merged_df = link_dfs[links[0]][['Frame', 'qpos', 'qvel']].copy()
merged_df.rename(columns={
    'qpos': f'qpos_{links[0]}',
    'qvel': f'qvel_{links[0]}'
}, inplace=True)

# Merge with remaining links
for link in links[1:]:
    link_data = link_dfs[link][['Frame', 'qpos', 'qvel']].copy()
    link_data.rename(columns={
        'qpos': f'qpos_{link}',
        'qvel': f'qvel_{link}'
    }, inplace=True)
    merged_df = pd.merge(merged_df, link_data, on='Frame')

In [ ]:
print(merged_df.head())

In [25]:
print(len(merged_df))

461


In [26]:
merged_df.to_csv(data_dir + 'kuka_formatted2.csv', index=False)